In [7]:
from click import prompt
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Tools


In [2]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

# 빌트인 도구(built-in tools)

## Python REPL 도구

In [3]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성
python_tool = PythonREPLTool()

In [4]:
# 파이썬 코드를 실행하고 결과를 반환
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)

# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
         "Return only the code, no intro, no chatty, no markdown, no code block, no nothing. Just the code."),
        ("human", "{input}")
    ]
)

# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [6]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE:
import random

def generate_lotto_numbers():
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
[12, 25, 27, 28, 32, 37]



## TavilySearchResults

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_result=6,
    include_answer=True,
    include_raw_content=True,
    include_domains=["github.io", "wikidocs.net"]
)

In [9]:
# 도구 실행
tool.invoke({"query": "LangChain Tools에 대해서 알려주세요."})

[{'title': 'Langchain | SynapseML - GitHub Pages',
  'url': 'https://microsoft.github.io/SynapseML/docs/Explore+Algorithms/OpenAI/Langchain/',
  'content': 'LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs). Chains in LangChain go beyond just a single LLM call and are sequences of calls (can be a call to an LLM or a different utility), automating the execution of a series of calls and actions. ... from langchain. tools',
  'score': 0.7961819},
 {'title': 'LangChain Fundamentals :: LangChain AI - esseai.github.io',
  'url': 'https://esseai.github.io/langchain_project_handbook/fundamentals/index.html',
  'content': 'In practical projects, deploying a large language model (LLM) as a service typically involves launching it in a Docker container, or Kubernetes. Hence, having familiarity with containers would also be advantageous.\nLangChain is an open-source tool with a Python and JavaScript codebase. However, 

## 추가된 TavilySearch 도구 (커스텀 구현)

In [10]:
from langchain_teddynote.tools.tavily import TavilySearch

# 기본 예제
tavily_tool = TavilySearch()

# include_domains 사용 예제
# 특정 도메인만 포함하여 검색
tavily_tool_with_domains = TavilySearch(include_domains=["github.io", "naver.com"])

# exclude_domains 사용 예제
# 특정 도메인을 제외하고 검색
tavily_tool_exclude = TavilySearch(exclude_domains=["ads.com", "spam.com"])

# 다양한 파라미터를 사용항 검색 예제
result1 = tavily_tool.search(
    query="유튜버 테디노트에 대해서 알려줘",
    search_depth="advanced",
    topic="general",
    days=7,
    max_results=10,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    format_output=True
)

# 뉴스 검색 예제
result2 = tavily_tool.search(
    query="최신 AI 기술 동향",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    topic="news",  # 뉴스 주제
    days=3,  # 최근 3일 내 결과
    max_results=5,  # 최대 5개 결과
    include_answer=False,  # 답변 미포함
    include_raw_content=False,  # 원본 콘텐츠 미포함
    include_images=False,  # 이미지 미포함
    format_output=True,  # 결과 포맷팅
)

# 특정 도메인 포함 검색 예제
result3 = tavily_tool_with_domains.search(
    query="파이썬 프로그래밍 팁",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    max_results=3,  # 최대 3개 결과
)

# 특정 도메인 제외 검색 예제
result4 = tavily_tool_exclude.search(
    query="건강한 식단",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    days=30,  # 최근 30일 내 결과
    max_results=7,  # 최대 7개 결과
)

# 결과 출력
print("기본 검색 결과:", result1)
print("뉴스 검색 결과:", result2)
print("특정 도메인 포함 검색 결과:", result3)
print("특정 도메인 제외 검색 결과:", result4)

기본 검색 결과: ['<document><title>teddylee777/QA-Dataset-mini · Datasets at Hugging Face</title><url>https://huggingface.co/datasets/teddylee777/QA-Dataset-mini</url><content>테디노트 유튜브 채널에 대해서 알려주세요. 테디노트(TeddyNote)는 데이터 분석, 머신러닝, 딥러닝 등의 주제를 다루는 유튜브 채널입니다. 이 채널을 운영하는 이경록님은 데이터 분석과 인공지능에 대한 다양한 강의를 제공하며, 초보자도 쉽게 따라할 수 있도록</content></document>', '<document><title>테디노트 TeddyNote | 두런 - DoLearn | 유튜브 채널소개</title><url>https://www.dolearn.ai/index.php/etc/user_introduction?type=youtuber&channel_id=UCt2wAAXgm87ACiQnDHQEW6Q</url><content>프로필 이미지. 테디노트 TeddyNote 유튜브 채널. 데이터 분석, 머신러닝, 딥러닝에 대한 내용을 다룹니다. 저는 머신러닝/딥러닝의 대중화를 꿈꿉니다. === 텐서플로우</content></document>', "<document><title>[지금 무료][테디노트] 한 방으로 끝내는 파이썬Python (전자책 포함) 강의 | 런어데이 - 인프런</title><url>https://www.inflearn.com/course/테디노트-파이썬</url><content>📖 테디노트 소개. 안녕하세요, 데이터 분석 블로그와 유튜브 채널 ' 테디노트 '를 운영하고 있는 이경록입니다. 제가 처음 파이썬을 공부할 당시에는 교육기관이나 자료가 많지 않아서 독학을 하는데 어려움이 많았습니다. 때문에 여러 시행착오도 겪고, 유튜브 온라인 강의, 그리고 전문가의</content></document>", '<document><title>\'테디의

## Image 생성 도구 (DALL-E)

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
     "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke({"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"})

# 이미지 프롬프트 출력
print(image_prompt)

A neo-classical painting depicting a group of people engaging with smartphones, set in a grand, classical environment reminiscent of ancient Greece. The individuals are dressed in elegant, flowing robes similar to those worn in classical art, embodying ideals of beauty and harmony. They are standing in a lush garden with marble statues and ornate fountains in the background. The expressions on their faces reflect a mix of fascination and distraction as they gaze at their devices, contrasting the timeless beauty of their surroundings with modern technology. Soft, dramatic lighting enhances the scene, creating a sense of depth and highlighting the juxtaposition of past and present. The overall composition should evoke a sense of irony, inviting viewers to reflect on the relationship between humanity and technology in a classical artistic style.


In [14]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

# DALL-E APi 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

# 질문
query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시
Image(url=image_url, width=500)

# 사용자 정의 도구(Custom Tool)

In [15]:
from langchain.tools import tool

# 데코레이터를 사용하여 함수를 도구로 변환
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [16]:
# 도구 실행
add_numbers.invoke({"a": 1, "b": 2})

3

In [17]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

# 구글 뉴스기사 검색 도구

In [18]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [19]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiggFBVV95cUxNczd0b2VjV3Y3LUJzWHpZazBPdF84UXhBNGhVUVlIT196b01HNFFfbWVZZlJZTFJpUXVaU0J4UE9QdjNWZFZuazdvXzdIM3VrUy1jVGxJdUxIMDF4R3QweFAtcVdrcVRWbzBIMnJTajJQVTJ5SmtTbXhVUHg1TnY3TzZR?oc=5',
  'content': '[포토] 고이 잠드소서…‘가난한 이들의 벗’ 프란치스코 교황 선종 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE45QUJRVHlmdnhaUDI5a3Z6N0k0UWdhcEN1VHVFMTJfaXZsaThnX1AwOE1Pdk5fSkI3cG1iOTk0cXVoY2tUNW91SFI5WF9rU2tCSEl0Y0ZzTVg0di1va1JlV1ZWQ3JZUHF0NUE?oc=5',
  'content': '"몰염치" "뻐꾸기" "눈썹 문신"... 막말과 인신공격에 얼룩진 국민의힘 경선 - 한국일보'},
 {'url': 'https://news.google.com/rss/articles/CBMidEFVX3lxTE1kTG5mMGpCSW5RUWN6bks5OS1zbkx1VFdoM1RGcnRCVWNtbHBkSjNTSllmSlBzZF81djFlME5HeW9SV1lDY0NhRTRyUGtRUHltMWxzX3dYcFFPYU90d2NKUU5HTmMzUXVCYlg4Um0yMl9hT1lx?oc=5',
  'content': '대선 앞 또 등장한 ‘세종 행정수도’…개헌 걸림돌 여전 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFB4TjhKUXpVVFUwdTVhSmIwQjdXYkQyalR6QWVUeVY3UHBBNHVRUTByM3QyWTZhUVdocU5NOHRJUklPa3Ftc0VSVF9GWDZfZEJlU0NhS1

In [20]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE1MckRhRy1VdDdHMXhBYUhIQWJrYXhGR2IyckFKUGhLSTlneFZCNnRPZlJCbjlDNDZCbGZjZW00cXE1VEN0M19TUDdQY0owVTRUeFp3bHhMRnp1QdIBVEFVX3lxTE1fLTI5bUVQdk5NZTZmWEkyRUhuM1RTWHJzSUZqQkNDSmNKSVlSUnNIT0FSanN3X2xOa25iYmRPakE3amVfd1hDOHZxSnZBVmswTTZKbA?oc=5',
  'content': '퓨리오사AI 투자 대박…TS·DSC인베스트 급등 - 한국경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiZkFVX3lxTE9HWEhnTHFRV3V1Tnlza1dGay1PeGdGdkhXQ3hwNHBMWEt4Y3MzYS1fdkZsQWZZVlhwWFZmeHlaWkhRSklsXzlRdTg3Rk9zaHh4MVltazY5QkRpQktUUEc4eGdESGo2Z9IBbkFVX3lxTFA0Um1fM3RQbVpGWGxIQy1yQmlxSkd4dkp4OW4wNDNPaWlzTVV1blRvVzh2cklucnpkalBpeUNnUlkzMzRVbDA0MFFudHM4Tzhvc3R5QTk4VnlYb1RXT0kxUWVFUllSTE4wdVVRLXBB?oc=5',
  'content': '이재명이 만난 퓨리오사AI에 투자금 쏠리자 창업투자株 급등 - 머니투데이'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE9rbUNLdjJsMGZSYUNxSE1xNDhwTGRZNlAtaks3TVQ2ekZNc3RrZU9uWUVURWxqZDE0bzgwbmV5Q1FvaUN1M1pIdEpXZHJZSWlEdGV2RmN2a1VpUdIBVEFVX3lxTE5PTEF6MElNZE13S2IzRDBLMU1wdUE4VUpmb2t3QWxNbUZPM0c0bFI4WlM0N3hl

In [21]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [22]:
# 실행 결과
search_keyword.invoke({"query": "AI 투자"})

AI 투자


[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE1MckRhRy1VdDdHMXhBYUhIQWJrYXhGR2IyckFKUGhLSTlneFZCNnRPZlJCbjlDNDZCbGZjZW00cXE1VEN0M19TUDdQY0owVTRUeFp3bHhMRnp1QdIBVEFVX3lxTE1fLTI5bUVQdk5NZTZmWEkyRUhuM1RTWHJzSUZqQkNDSmNKSVlSUnNIT0FSanN3X2xOa25iYmRPakE3amVfd1hDOHZxSnZBVmswTTZKbA?oc=5',
  'content': '퓨리오사AI 투자 대박…TS·DSC인베스트 급등 - 한국경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiZkFVX3lxTE9HWEhnTHFRV3V1Tnlza1dGay1PeGdGdkhXQ3hwNHBMWEt4Y3MzYS1fdkZsQWZZVlhwWFZmeHlaWkhRSklsXzlRdTg3Rk9zaHh4MVltazY5QkRpQktUUEc4eGdESGo2Z9IBbkFVX3lxTFA0Um1fM3RQbVpGWGxIQy1yQmlxSkd4dkp4OW4wNDNPaWlzTVV1blRvVzh2cklucnpkalBpeUNnUlkzMzRVbDA0MFFudHM4Tzhvc3R5QTk4VnlYb1RXT0kxUWVFUllSTE4wdVVRLXBB?oc=5',
  'content': '이재명이 만난 퓨리오사AI에 투자금 쏠리자 창업투자株 급등 - 머니투데이'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE9rbUNLdjJsMGZSYUNxSE1xNDhwTGRZNlAtaks3TVQ2ekZNc3RrZU9uWUVURWxqZDE0bzgwbmV5Q1FvaUN1M1pIdEpXZHJZSWlEdGV2RmN2a1VpUdIBVEFVX3lxTE5PTEF6MElNZE13S2IzRDBLMU1wdUE4VUpmb2t3QWxNbUZPM0c0bFI4WlM0N3hl